## **Imports**

First, it's necessary to import the libraries that will be used in the script below and the dataset obtained from the [website](https://sistemas.anatel.gov.br/se/public/view/b/licenciamento.php) of the Brazilian National Telecommunications Agency.

### **Libraries**

In [1]:
import pandas as pd
import numpy as np
import folium
from folium.raster_layers import ImageOverlay

### **Dataset**

The dataset columns used in this scripts are:
* **'NumEstacao':** unique number that characterizes a station.
* **'DataValidade':** expiration date of the radio frequency associated with the station.
* **'Latitude':** the station's latitude coordinate. 
* **'Longitude':** the station's longitude coordinate. 

In [2]:
#The variable path must contain a string with the path to the dataset used.
path = '../datasets/natal_04_11_2024.csv' #2024
year = '2024'

#path = '../datasets/natal_03_31_2025.csv' #2025
#year = '2025'

df = pd.read_csv(path, encoding='unicode_escape')

## **Data Adjustment**

Some columns are not in the format usable for the script, so they have been fixed. Rows whose 'DataValidade' value is before the year the work was developed need to be removed.

In [3]:
#Conversion of the column 'DataValidade' to the datetime64[ns] type.
df['DataValidade'] = pd.to_datetime(df['DataValidade'])

##Excludes 'DataValidade' lines with values before January 1st of the selected Year.
df = df[df['DataValidade'] > pd.to_datetime(str(year)+'-01-01', format='%Y-%m-%d')]

#Removes duplicates based on the 'NumEstacao' column.
df_map = df.drop_duplicates(subset=['NumEstacao'])

## **Plot**

In [4]:
#Create a map centered at the average of the coordinates
average_latitude = df['Latitude'].mean()
average_longitude = df['Longitude'].mean()
stations_map = folium.Map(location=[average_latitude, average_longitude], zoom_start=14, tiles='Esri.WorldImagery')

#Create circles for each station
R = 100  #Radius of the circles (in meters)
theta = np.linspace(0, 2*np.pi, 100)

for _, row in df_map.iterrows():
    lat, lon = row['Latitude'], row['Longitude']

    #Convert meters to degrees (approximately)
    delta_lat = R / 111320  #1 degree of latitude ≈ 111.32 km
    delta_lon = R / (111320 * np.cos(np.radians(lat)))

    #Coordinates of the circle
    circle_lats = lat + delta_lat * np.sin(theta)
    circle_lons = lon + delta_lon * np.cos(theta)
    coordinates = list(zip(circle_lats, circle_lons))

    #Add the circle to the map
    folium.Polygon(locations=coordinates, color='yellow', fill=True, fill_opacity=0.2).add_to(stations_map)

    # Add a marker for the central point
    folium.Marker([lat, lon], popup=f"Station: {row['NumEstacao']}, {row['Latitude']} {row['Longitude']}", icon=folium.Icon(color='red')).add_to(stations_map)

#Display the map
stations_map